In [1]:
!mkdir -p /scratch/sagarsj42/torch-cache
!mkdir -p /scratch/sagarsj42/transformers

import os
os.chdir('/scratch/sagarsj42')
os.environ['TORCH_HOME'] = '/scratch/sagarsj42/torch-cache'
os.environ['TRANSFORMERS_CACHE'] = '/scratch/sagarsj42/transformers'
os.environ['TOKENIZERS_PARALLELISM'] = 'false'

In [2]:
from functools import partial

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from datasets import load_dataset, load_metric

from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import Trainer, TrainingArguments

/home2/sagarsj42/miniconda3/lib/python3.9/site-packages/apex/pyprof/__init__.py:5: FutureWarning: pyprof will be removed by the end of June, 2022
  warnings.warn("pyprof will be removed by the end of June, 2022", FutureWarning)


In [3]:
TRAIN_DATA_FILE = 'irse.train.csv'
DEV_DATA_FILE = 'irse.dev.csv'
TEST_DATA_FILE = 'irse.test.csv'
GIVEN_TEST_FILE = 'irse.given-test.csv'
MODEL_KEY = 'bert-base-uncased'
EXP_NAME = 'irse-bert'

TRAIN_BATCH_SIZE = 4
EVAL_BATCH_SIZE = 16
ACCUMULATE_GRAD_STEPS = 2
N_EPOCHS = 5
LEARNING_RATE = 6e-5
SCHEDULER_TYPE = 'cosine'
LR_WARMUP_RATIO = 0.4
LOG_STEPS = 50
SEED = 43419

In [4]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_KEY)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_KEY)

tokenizer, model

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

(PreTrainedTokenizerFast(name_or_path='bert-base-uncased', vocab_size=30522, model_max_len=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}),
 BertForSequenceClassification(
   (bert): BertModel(
     (embeddings): BertEmbeddings(
       (word_embeddings): Embedding(30522, 768, padding_idx=0)
       (position_embeddings): Embedding(512, 768)
       (token_type_embeddings): Embedding(2, 768)
       (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
       (dropout): Dropout(p=0.1, inplace=False)
     )
     (encoder): BertEncoder(
       (layer): ModuleList(
         (0): BertLayer(
           (attention): BertAttention(
             (self): BertSelfAttention(
               (query): Linear(in_features=768, out_features=768, bias=True)
               (key): Linear(in_features=768, out_features=768, bias=True)
               (val

In [5]:
data_files = {
    'train': TRAIN_DATA_FILE,
    'dev': DEV_DATA_FILE,
    'test': TEST_DATA_FILE,
    'giventest': GIVEN_TEST_FILE
}
ds = load_dataset('csv', data_files=data_files)

ds

Using custom data configuration default-45d70fe36903500c
Reusing dataset csv (/home2/sagarsj42/.cache/huggingface/datasets/csv/default-45d70fe36903500c/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a)


  0%|          | 0/4 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['Comments', 'Surrounding Code Context', 'Class', 'label'],
        num_rows: 5354
    })
    dev: Dataset({
        features: ['Comments', 'Surrounding Code Context', 'Class', 'label'],
        num_rows: 595
    })
    test: Dataset({
        features: ['Comments', 'Surrounding Code Context', 'Class', 'label'],
        num_rows: 678
    })
    giventest: Dataset({
        features: ['Comments', 'Surrounding Code Context', 'Class', 'label'],
        num_rows: 1001
    })
})

In [6]:
def tokenizer_func(batch, tokenizer):
    text = [batch['Comments'][i] + tokenizer.sep_token + batch['Surrounding Code Context'][i] 
        for i in range(len(batch['Comments']))]
    tok = tokenizer(text, max_length=tokenizer.model_max_length, truncation=True, 
        padding=True, return_attention_mask=True)

    return tok

In [7]:
tokenizer_partial = partial(tokenizer_func, tokenizer=tokenizer)
ds_tok = ds.map(tokenizer_partial, batched=True, load_from_cache_file=False)

ds_tok

  0%|          | 0/6 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

DatasetDict({
    train: Dataset({
        features: ['Comments', 'Surrounding Code Context', 'Class', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 5354
    })
    dev: Dataset({
        features: ['Comments', 'Surrounding Code Context', 'Class', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 595
    })
    test: Dataset({
        features: ['Comments', 'Surrounding Code Context', 'Class', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 678
    })
    giventest: Dataset({
        features: ['Comments', 'Surrounding Code Context', 'Class', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1001
    })
})

In [8]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    scores = logits[:, -1]
    
    acc_metric = load_metric('accuracy')
    f1_metric = load_metric('f1')
    mcc_metric = load_metric('matthews_correlation')
    roc_metric = load_metric('roc_auc')

    metrics_dict = {
        'accuracy': acc_metric.compute(predictions=predictions, references=labels)['accuracy'],
        'f1_score': f1_metric.compute(predictions=predictions, references=labels)['f1'],
        'matthews_cc': mcc_metric.compute(predictions=predictions, references=labels)['matthews_correlation'],
        'roc_auc_score': roc_metric.compute(prediction_scores=scores, references=labels)['roc_auc'],
    }

    return metrics_dict

In [9]:
training_args = TrainingArguments(
    max_steps=100,
    output_dir=EXP_NAME,
    run_name=EXP_NAME,
    overwrite_output_dir=True,
    do_train=True,
    do_eval=True,
    per_device_train_batch_size=TRAIN_BATCH_SIZE,
    per_device_eval_batch_size=EVAL_BATCH_SIZE,
    gradient_accumulation_steps=ACCUMULATE_GRAD_STEPS,
    num_train_epochs=N_EPOCHS,
    learning_rate=LEARNING_RATE,
    lr_scheduler_type=SCHEDULER_TYPE,
    warmup_ratio=LR_WARMUP_RATIO,
    evaluation_strategy='epoch',
    logging_steps=LOG_STEPS,
    save_strategy='epoch',
    metric_for_best_model='f1',
    greater_is_better=True,
    load_best_model_at_end=True,
    no_cuda=False,
    seed=SEED,
    fp16=False,
    dataloader_drop_last=False
)

training_args

TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=False,
do_train=True,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=None,
evaluation_strategy=IntervalStrategy.EPOCH,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
gradient_accumulation_steps=2,
gradient_checkpointing=False,
greater_is_better=True,
group_by_length=False,
half_precision_backend=auto,
hub_model_id=None,
hub_strategy=HubStrategy.EVERY_SAVE,
hub_token=<HUB_TOKEN>,
ignore_data_skip=False,
label_names=None,
label_smoothing_factor=0.0,
learning_rate=6e-05,
length_column_name=length,
load_best_model_at_end=True,
local_rank=-1,
log_level=-1,
log_level_replica=-1,
log_on_each_node=True,
logg

In [10]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=ds_tok['train'],
    eval_dataset=ds_tok['dev'],
    compute_metrics=compute_metrics
)

trainer

RuntimeError: CUDA error: out of memory
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.

In [ ]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: Surrounding Code Context, Comments, Class. If Surrounding Code Context, Comments, Class are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
/home2/sagarsj42/miniconda3/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 5354
  Num Epochs = 1
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 2
  Total optimization steps = 100
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"

RuntimeError: CUDA out of memory. Tried to allocate 20.00 MiB (GPU 0; 10.76 GiB total capacity; 1.46 GiB already allocated; 10.56 MiB free; 1.51 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
preds, labels, metrics = trainer.predict(ds_tok['test'])

preds.shape, labels.shape, metrics

The following columns in the test set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: Surrounding Code Context, Comments, Class. If Surrounding Code Context, Comments, Class are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 678
  Batch size = 8


((678, 2),
 (678,),
 {'test_loss': 1.445356011390686,
  'test_accuracy': 0.6887905604719764,
  'test_f1_score': 0.4373333333333333,
  'test_matthews_cc': 0.3443045997673132,
  'test_roc_auc_score': 0.719450097227875,
  'test_runtime': 16.1345,
  'test_samples_per_second': 42.022,
  'test_steps_per_second': 5.268})

In [ ]:
gpreds, glabels, gmetrics = trainer.predict(ds_tok['giventest'])

gpreds.shape, glabels.shape, gmetrics

The following columns in the test set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: Surrounding Code Context, Comments, Class. If Surrounding Code Context, Comments, Class are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 1001
  Batch size = 8


((1001, 2),
 (1001,),
 {'test_loss': 1.0055490732192993,
  'test_accuracy': 0.7832167832167832,
  'test_f1_score': 0.44783715012722647,
  'test_matthews_cc': 0.4012322064192643,
  'test_roc_auc_score': 0.7382100829560363,
  'test_runtime': 19.0209,
  'test_samples_per_second': 52.626,
  'test_steps_per_second': 6.624})